### Generate rpoB Database

In [80]:
import pandas as pd 
import numpy as np 
from Bio import SeqIO
import random
df = pd.read_table('../data/rpoB/nuccore_result_table.txt')
df.head()
random.seed(10)


### Remove Unaligned Species 
unaligned = ["AF060365.1","JQ314438.1","MF145297.1","GU362494.1","GU362473.1", "AF060364.1", "GU362478.1","AF060363.1","AF060361.1","GU362487.1","AF060355.1","MF471868.1"
        , "AF060362.1", "AF060359.1","AF060356.1","DQ985207.1", "DQ985218.1", "KF224998.1" ]
df = df[~df['Accession'].isin(unaligned)]



## Filter for Culture Codes
cultures = df['Accession'].str.contains('AF0603|MF1452|DQ9852|GU3624|KF2249|MG9950|JN3153|JQ3144|KJ6837')
df = df[~cultures]
len(df)

1590

In [81]:
### Remove Unverified and unclassified Bacteria
list_names = df['Name']
g = [x.split(' ',2)[0] for x in list_names]
matching = [s == "Mycobacterium" for s in g]
df_filtered = df[matching]
len(df_filtered)

1507

In [82]:
list_names = df_filtered['Name']
g = [x.split(' ',2)[1] for x in list_names]
matching = [s != "sp." for s in g]
df_filtered = df_filtered[matching]
len(df_filtered)

1342

In [83]:
list_names = df_filtered['Name']
g = [x.split(' ',2)[1] for x in list_names]
matching = [s != "massiliense" for s in g]
df_filtered = df_filtered[matching]
len(df_filtered)

1219

In [84]:
### Split String Name
list_names = df_filtered['Name']
g = [x.split(' ',2)[0] for x in list_names]
s = [x.split(' ',2)[1] for x in list_names]
y = list(zip(g,s))
y = [list(x) for x in y]
z = [' '.join(x) for x in y]

In [85]:
df_filtered['Species'] = z
df_filtered.head()

,Name,Length,Accession,GI,Species
0,Mycobacterium bouchedurhonense strain 4355387 ...,711,EF584445.1,157851024,Mycobacterium bouchedurhonense
1,Mycobacterium marseillense strain 5356591 RpoB,711,EF584434.1,157851003,Mycobacterium marseillense
2,Mycobacterium bacteremicum strain ATCC 25791,638,FJ172329.1,207690910,Mycobacterium bacteremicum
3,Mycobacterium marseillense strain 62863 RpoB,711,EF584444.1,167540464,Mycobacterium marseillense
4,Mycobacterium timonense strain 3256799 RpoB,711,EF584435.1,157851005,Mycobacterium timonense


In [86]:
df2 = df_filtered[['Name','Species']].groupby('Species').agg(['count'])
df2.columns = ['Abundance']
len(df2)

136

In [87]:
bool1 = df_filtered['Name'].str.contains('ATCC')
bool2 = df_filtered['Name'].str.contains('DSM')
bool3 = df_filtered['Name'].str.contains('CCUG')
bool4 = df_filtered['Name'].str.contains('ASIN')
bool5 = df_filtered['Name'].str.contains('QIA')
bool6 = df_filtered['Name'].str.contains('GTC')

df_filtered['Cultures'] = bool1|bool2|bool3|bool4|bool5|bool6
df_filtered['Cultures'] = df_filtered['Cultures'].astype(int)

In [88]:
df2_s = df2[df2['Abundance'] > 2]
df2_s.head()
test = list(df2_s.index)

## Keep everything in this df
df_auto_pick = df_filtered[~df_filtered['Species'].isin(test)]
small_counts = list(df_auto_pick['Accession'])

In [89]:
## Randomly select two species (Keep Cultures)
df_mult = df_filtered[df_filtered['Species'].isin(test)]
len(df_mult)

df3 = df_mult[['Name','Species']].groupby('Species').agg(['count'])
df3.columns = ['Abundance']
df3.head()

,Abundance
Species,
Mycobacterium abscessus,129
Mycobacterium alvei,12
Mycobacterium arceuilense,3
Mycobacterium arupense,20
Mycobacterium aurum,5


In [90]:
## Count Abundance of Culture Codes
df4 = df_mult[['Species', 'Cultures']].groupby('Species').sum()
df4.columns = ['Abundance']
df4.head()

,Abundance
Species,
Mycobacterium abscessus,4
Mycobacterium alvei,0
Mycobacterium arceuilense,0
Mycobacterium arupense,1
Mycobacterium aurum,1


In [91]:
### Multiples with 0
zeros = df4[df4['Abundance'] < 1]
zeros = list(zeros.index)


### Multiples with 1
ones = df4[df4['Abundance'] == 1]
ones = list(ones.index)


### Multiples with 2
twos = df4[df4['Abundance'] == 2]
twos = list(twos.index)

### Multiples with Greater than 2
greater = df4[df4['Abundance'] > 2]
greater = list(greater.index)

In [92]:
## Select Accession of Organism with two cultures 
twos_A = df_mult[df_mult['Species'].isin(twos)]
twos_A = twos_A[twos_A['Cultures'] == 1]
twos_A = list(twos_A['Accession'])
twos_list = twos_A

In [93]:
## Greater than twos
import random
my_acc = []
greatera = df_mult[df_mult['Species'].isin(greater)]
greatera = greatera[greatera['Cultures'] == 1]
for name, group in greatera.groupby('Species'):
    my_list = list(group['Accession'])
    one, two = random.sample(my_list,2)
    my_acc.append(one)
    my_acc.append(two)
   
greater_list = my_acc
len(greater_list)

24

In [94]:
## Ones
my_ones = []
onesa = df_filtered[df_filtered['Species'].isin(ones)]
onesa_1 = onesa[onesa['Cultures'] == 1]
onesa_1 = list(onesa_1['Accession'])
onesa = onesa[onesa['Cultures'] == 0]
for name, group in onesa.groupby('Species'):
    my_list = list(group['Accession'])
    one = random.sample(my_list, 1)
    my_ones.append(one)

my_ones = [item for sublist in my_ones for item in sublist]
ones_list = my_ones+onesa_1

In [95]:
## Zeros
my_zeros = []
zeroa = df_mult[df_mult['Species'].isin(zeros)]
zeroa = zeroa[zeroa['Cultures'] == 0]
for name, group in zeroa.groupby('Species'):
    my_list = list(group['Accession'])
    one, two = random.sample(my_list,2)
    my_zeros.append(one)
    my_zeros.append(two)
    
zeros_list = my_zeros
len(zeros_list)

54

In [96]:
## Combine all Accessions
Combined_Accessions = zeros_list + ones_list + twos_list + greater_list + small_counts
df5 = df_filtered[df_filtered['Accession'].isin(Combined_Accessions)]
len(df5)

224

In [97]:
writer = pd.ExcelWriter('Counts_of_Species.xlsx')
df5.to_excel(writer, "Sheet2")
writer.save()

In [98]:
from Bio import SeqIO

with open('../rpoB.fasta', 'w') as out_file:
    for fasta in SeqIO.parse('../data/rpoB/sequence.fasta','fasta'):
        name, sequence = fasta.id, fasta.seq
        if name in Combined_Accessions:
            SeqIO.write(fasta, out_file, 'fasta')

### Check Accession Overlap

In [1]:
from Bio import SeqIO
from collections import defaultdict
records=list(SeqIO.parse("../Databases/rpoB/rpob_region.fasta", "fasta"))

In [2]:
from collections import OrderedDict

d = OrderedDict()
for record in records:
    if record.seq in d:
        d[record.seq].append(record)
    else:
        d[record.seq] = [record]
        
len(d)

/Users/stronglab2/anaconda2/lib/python2.7/site-packages/Bio/Seq.py:155: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


197

In [3]:
q = {k: v for k, v in d.iteritems() if len(v)> 1}

keep = []

for seq, record_set in q.iteritems():
    
    my_list = []
    for record in record_set:
        
        my_list.append(record.description.split(' ')[2])
    my_list = list(set(my_list))
    if len(my_list) > 1:
        keep.append(True)
    else:
        keep.append(False)

In [4]:
weird = dict()
for i in range(len(keep)):
    if keep[i]==True:
        weird[q.keys()[i]] = q.values()[i]
len(weird)

3

In [5]:
for seq, record_set in weird.iteritems():
    print seq + ': (' + str(len(record_set)) + ')'
    for record in record_set:
        print '    ' + record.description

ATCTTCGGCGAGAAGGCCCGCGAGGTCCGCGACACGTCGCTGAAGGTGCCGCACGGCGAGTCCGGCAAGGTCATCGGCATCCGGGTGTTCTCCCGCGAGGACGACGACGAGCTGCCCGCCGGCGTCAACGAGCTGGTCCGCGTGTACGTGGCCCAGAAGCGCAAGATCTCCGACGGCGACAAGCTGGCCGGCCGGCACGGCAACAAGGGTGTGATCGGCAAGATCCTGCCCGTCGAGGACATGCCGTTCCTGCCGGACGGCACACCGGTGGACATCATCCTCAACACCCACGGGGTGCCGCGACGGATGAACATCGGCCAGATCCTGGAAACCCACCTGGGCTGGTGCGCGCACAGCGGCTGGAAGGTCGAGGCCCGACTGGGCCGCCAAGCTGCCGGAGGAGCTGCTGGAGGCCGAGCCGAACAAGATCGTCTCGACGCCGGTGTTCGACGGTGCCCGCGAGGAGGAGCTGCAGGGGCTGCTGTCCTCGACGCTGCCCAACCGCGACGGCGACGTGCTGGTCGACGGCGACGGCAAGGCGATCCTGTACGACGGGCGCTCCGGTGAGCCGTTCCCGTACCCGGTGACGACCGGCTACATGTACATCATGAAGCTGCACCACCTGGTGGACGACAAGATCCACGCCCGCTCCACCGG: (3)
    JF346871.1 Mycobacterium celatum strain ATCC 51130 RNA polymerase beta subunit (rpoB) gene, partial cds
    JN866833.1 Mycobacterium kyorinense strain DSM 45166 RNA polymerase beta subunit (rpoB) gene, partial cds
    JQ744020.1 Mycobacterium kyorinense strain HF1629 RNA polymerase subunit B (rpoB) gene, complete cds
ATCTTCGGCGAGAA

In [30]:
count = 0
for seq, record_set in weird.iteritems():
    for record in record_set:
        count += 1
count

2

### Cluster Sequences

In [6]:
ids_to_remove = []
for seq, record_set in weird.iteritems():
    for record in record_set:
        ids_to_remove.append(record.id)

In [7]:
replace_seq = []
for i in weird.iterkeys():
    replace_seq.append(i)

replace_groups = ['MH: kyorinense | celatum', 'MH: intracellulare| paraintracellulare', 'MH: tuberculosis | pinnipedii | microti | africanum']

In [11]:
from Bio import SeqIO

fasta_sequences = SeqIO.parse(open('../Databases/rpoB/rpob_region.fasta'),'fasta')
new_items = []
for fasta in fasta_sequences:
    name = fasta.id
    if name in ids_to_remove:
        continue
    else:
        new_items.append(fasta)
        
for item in range(len(replace_groups)):
    val = SeqIO.SeqRecord(id=replace_groups[item], seq=replace_seq[item])
    new_items.append(val)
SeqIO.write(new_items, "../Databases/rpoB/rpoB_clustered_region.fasta", "fasta")

216